In [1]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta, date
import sys

'''This script produces a file which shows availability. Each row is a booking
date, and each col is a (room type, arrival week, arrival day of week, stay length) tuple'''

df_data = pd.read_csv('../../../data/cabot_data/clean_base/2018_clean.csv',
                       parse_dates = ['LOOK_DATE', 'ARRIVAL', 'DEPART'],
                       infer_datetime_format = True,
                       index_col = 'group_id')
df_data = df_data.reset_index()

df_caps = pd.read_csv('../../../data/cabot_data/clean_base/capacities.csv', index_col='UNIT')

In [2]:
# add stay len col to df_data
df_data['stay_len'] = (df_data['DEPART'] - df_data['ARRIVAL']).dt.days
# anything greater than 4 is 4
df_data['stay_len_clean'] = df_data['stay_len'].apply(lambda x: 4 if x > 4 else x)

In [3]:
# helper vars
look_start = df_data['LOOK_DATE'].min()
look_end = df_data['LOOK_DATE'].max()
look_range = pd.date_range(look_start, look_end)

ssn_start = df_data['ARRIVAL'].min()
ssn_end = df_data['ARRIVAL'].max()
ssn_range = pd.date_range(ssn_start, ssn_end)

# get season week range
ssn_weeks = ssn_range.week.drop_duplicates().tolist()

# get max periods per day from transactions script
periods_per_day = (df_data[df_data['CANCEL_INDICATOR'] == 0] # look at bookings only
                   .drop(['CANCEL_INDICATOR'], axis=1)
                   .groupby('group_id').first() # collapse groups
                   .sort_values('LOOK_DATE')
                   .groupby(by=['LOOK_DATE']).count()
                   .max()[0])
intraday_range = range(1, periods_per_day+1)

# list of days in week
week_days = [1, 2, 3, 4, 5, 6, 7]

# list of stay lengths (4 refers to 4 days or more)
stay_lens = [1, 2, 3, 4]

# room type list
df_grouped_types = df_data.groupby(by='UNIT').count()
unit_list = df_grouped_types.index.tolist()

In [4]:
# initializing blank avail df
# create blank df
df_avail = pd.DataFrame(index=look_range,
                        columns=pd.MultiIndex.from_product([unit_list, ssn_weeks, week_days, stay_lens], 
                                                           names=['UNIT', 'week', 'dow', 'stay_len']))
df_avail = df_avail.fillna(0.0)

# add default capacities
for index, row in df_caps.iterrows():
    df_avail[index] = df_avail[index] + row['CAPACITY']

In [5]:
# helper load bar function
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()
total = len(df_data)

In [6]:
for index, row in df_data.iterrows():
    progress(index, total, status='Filling in availability')
    
    cap_change = 1/row['grp_size']
    
    # book arrive delta, must subtract capacity for all these dates
    ba_delta = pd.date_range(row['LOOK_DATE'], row['ARRIVAL'])
    
    # helper vars
    unit = row['UNIT']
    week = row['ARRIVAL'].week
    dow = row['ARRIVAL'].dayofweek + 1
    stay_len = row['stay_len']
    
    # subtract or add capacity for each day b/w look and arrive
    if row['CANCEL_INDICATOR'] == 0:
        for look_date in ba_delta:
            for n in range(0, stay_len):
                dow_temp = (n + dow-1) % 7 + 1 # points to day of week
                week_temp = (week + (n + dow - 1)//7)  # moves week forward 1 if stay crosses over into next week
                num_lengths_affected = min(stay_len - n, 4) # tracks which stay lengths are affected
                for length in range(0, num_lengths_affected):
                    df_avail.loc[look_date, (unit, week_temp, dow_temp, length+1)] -= cap_change
    else:
        for look_date in ba_delta:
            for n in range(0, stay_len):
                dow_temp = (n + dow-1) % 7 + 1
                week_temp = (week + (n + dow - 1)//7)
                num_lengths_affected = min(stay_len - n, 4)
                for length in range(0, num_lengths_affected):
                    df_avail.loc[look_date, (unit, week_temp, dow_temp, length+1)] -= cap_change

In [7]:
# create intraday template to cross join with avail df
df_intra_template = pd.DataFrame(index=intraday_range)
df_intra_template = df_intra_template.reset_index().rename(index=int, columns={'index':'intraday'})
df_intra_template

df_avail = df_avail.reset_index().rename(index=int, columns={'index':'look_date'})

# cross join
df_avail['key'] = 0
df_intra_template['key'] = 0
df_avail = df_avail.merge(df_intra_template, on='key', how='outer')

# tidy up avail
df_avail = df_avail.rename(index=str, columns={('look_date','','',''): 'look_date'})
df_avail = df_avail.set_index(['look_date', 'intraday'])
df_avail = df_avail.drop(['key', ('key', '', '', '')], axis=1)
df_avail.columns = pd.MultiIndex.from_tuples(df_avail.columns,
                                             names=['UNIT', 'week', 'dow', 'stay_length'])

# save
df_avail.to_csv('../../../data/cabot_data/sprint_2/avail_s2_raw.csv')

/Users/johngao/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (4 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [8]:
df_avail_t = pd.read_csv('../../../data/cabot_data/sprint_2/avail_s2_raw.csv',
                       index_col = [0,1],
                       header=[0,1,2,3])

In [9]:
# convert to a form that ipopt can understand

# reset to numbered rows
df_avail_t = df_avail_t.reset_index()
df_avail_t.index = df_avail_t.index + 1

# drop old indexes, make T new index
df_avail_t = df_avail_t.drop(['look_date', 'intraday'], axis=1)
df_avail_t.index = df_avail_t.index.rename('T')

# map values to 1s and 0s
df_avail_t = df_avail_t.applymap(lambda x: 0 if x <= 0 else 1)
df_avail_t.columns = df_avail_t.columns.tolist()

# map product tuples to numbers
prod_list = df_avail_t.columns.tolist()
prod_num_map = {prod_list[i]: i+1 for i in range(0, len(prod_list))}
prod_num_map = pd.Series(prod_num_map)

# save product mapping
prod_num_map.to_csv('../../../data/cabot_data/sprint_2/prod_num_map.csv')

/Users/johngao/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [10]:
# import col tuple to number map
prod_num_map = pd.read_csv('../../../data/cabot_data/sprint_2/prod_num_map.csv',
                           header=None,
                           index_col=[0,1,2,3])

prod_num_map = pd.Series(prod_num_map.iloc[:,0])
prod_num_map = prod_num_map.to_dict()

# map cols to numbers
df_avail_t.columns = prod_num_map.values()

In [11]:
df_avail_t.to_csv('../../../data/cabot_data/sprint_2/avail_s2.csv')

In [12]:
df_avail_view = pd.read_csv('../../../data/cabot_data/sprint_2/avail_s2.csv')

In [15]:
df_avail_t

,1,2,3,4,5,6,7,8,9,10,...,4891,4892,4893,4894,4895,4896,4897,4898,4899,4900
T,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
